In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score , mean_absolute_error, mean_squared_error , mean_absolute_percentage_error, median_absolute_error, mean_squared_log_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 


In [2]:
print (f"Numpy version:{np.__version__}")
print (f"Pandas version:{pd.__version__}")


Numpy version:1.23.2
Pandas version:1.4.3


In [3]:
def get_data_separated(dataframe, test_size=0.2, random_state=None):
    target_feature = ['ΔVC -p'] #+ ΔVC-m,ΔVC -p
    features_to_drop = ['substituent','molecular_formula','Name','canonical_smiles','ΔVC-m','cid']

    # Assuming 'target_feature' is the dependent variable (y)
    y = dataframe[target_feature]
    X = dataframe.drop(columns=features_to_drop + target_feature)
    X =X.fillna(0)

    # Split into training and testing sets, excluding data_val
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    # create the next sets , you split the temp data into halfs 
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=random_state
    )

    return X_train, X_val, X_test, y_train, y_val, y_test


In [4]:
def scaling_data(X_train,X_val,X_test):

    scaler = StandardScaler()

    scaler.fit(X_train)
    
    # filename = '../models/scaler_for_all_features.sav'
    # pickle.dump(scaler, open(filename, 'wb'))
    
    scaled_train = scaler.transform(X_train)
    scaled_test = scaler.transform(X_test)
    scaled_val = scaler.transform(X_val)
    return scaled_train,scaled_val,scaled_test

In [11]:
def save_metrics_results(model,X_test,y_test,tag):

    y_pred = model.predict(X_test)
    r2 = r2_score(y_pred=y_pred,y_true=y_test)
    MAE = mean_absolute_error(y_pred=y_pred,y_true=y_test) 
    MSE = mean_squared_error(y_pred=y_pred,y_true=y_test) 
    MAPE = mean_absolute_percentage_error(y_pred=y_pred,y_true=y_test)
    MedAE = median_absolute_error(y_pred=y_pred,y_true=y_test)
    # MSLE = mean_squared_log_error(y_pred=y_pred,y_true=y_test)

    results = {
        "R^2": r2,
        "MAE":MAE,
        "MSE":MSE,
        "MAPE":MAPE ,
        "MedAE":MedAE,
        # "MSLE":MSLE
    }
    mean_df = pd.DataFrame(data=results,index=[f"{tag}"])
    return mean_df

In [ ]:

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge, HuberRegressor, PassiveAggressiveRegressor, RANSACRegressor, ARDRegression, OrthogonalMatchingPursuit
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.isotonic import IsotonicRegression

def load_regression_models():
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge': Ridge(),
        'Lasso': Lasso(),
        'Elastic Net': ElasticNet(),
        'Bayesian Ridge': BayesianRidge(),
        'Huber Regressor': HuberRegressor(),
        'Passive Aggressive Regressor': PassiveAggressiveRegressor(),
        'RANSAC Regressor': RANSACRegressor(),
        'Isotonic Regression': IsotonicRegression(),
        'ARD Regression': ARDRegression(),
        'Orthogonal Matching Pursuit': OrthogonalMatchingPursuit(),
        'Decision Tree': DecisionTreeRegressor(),
        'Random Forest': RandomForestRegressor(),
        'Gradient Boosting': GradientBoostingRegressor(),
        'K-Nearest Neighbors': KNeighborsRegressor(),
        'Support Vector Regression': SVR(),
        'Stochastic Gradient Descent': SGDRegressor(),
        'Multi-layer Perceptron': MLPRegressor()
    }
    return models




In [ ]:
# Example usage:
regression_models = load_regression_models()
for name, model in regression_models.items():
    print(f"{name}: {model}")

In [6]:
rng = np.random.RandomState(42)
# data loading 
path = '/Users/barradd/Documents/GitHub/machine_learning_chem_RGS/data/pubchem_data_2023-12-05_343_samples.csv'
df_all_data = pd.read_csv(path)


In [7]:
X_train, X_val, X_test, y_train, y_val, y_test = get_data_separated(df_all_data,random_state=rng)
X_train, X_val, X_test = scaling_data(X_train, X_val, X_test)

In [20]:
def plot_the_r2(y_true, X_pred , model): #(model,X_test,y_test,tag)
    y_true = y_true.to_numpy().ravel()
    y_pred = model.predict(X_pred)
    plt.figure(figsize=(10, 4))

    plt.subplot(1, 2, 1)
    plt.scatter(y_true, y_pred, color='blue')
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], linestyle='--', color='red')
    plt.title('Real Values vs. Predicted Values')
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')

    # Plot Residuals
    residuals = y_true - y_pred

    plt.subplot(1, 2, 2)
    plt.scatter(y_true, residuals, color='green')
    plt.axhline(y=0, linestyle='--', color='red')
    plt.title('Residual Plot')
    plt.xlabel('Actual Values')
    plt.ylabel('Residuals')

    plt.tight_layout()
    plt.show()

In [ ]:
## create a loop that goes over all the list of algoritms 
## then use the fit function to train them
## the next step is to save the metrics per model 
## check the evaluation and plot the models that you think is relevant